In [1]:
from boink import libboink
from boink.data import load_ukhs
from boink.prometheus import Instrumentation
from boink.assembly import Assembler
from cppyy.gbl import std
from cppyy import gbl
import cppyy
import difflib

In [2]:
cppyy.cppdef('''
#include <exception>

template <typename T>
class TemplateThrows {
public:
    void throws(int i) {
        throw std::exception();
    }
};

template <typename T>
class TemplateOverloadedThrows {
public:
    void throws(int i) {
        throw std::exception();
    }

    void throws(double i) {
        throw std::exception();
    }
};

class OverloadedThrows {
public:

    void throws(double i) {
        throw std::exception();
    }
    
    void throws(int i) {
        throw std::exception();
    }



};

class RegularThrows {
public:

    void throws() {
        throw std::exception();
    }
    
};
''')

True

In [4]:
cppyy.cppdef('''

template <class Derived>
class Base {

int val;

public:

    int interface() {
        return derived().implementation(val);
    }

private:

    Base(int val) : val(val) {

    }

    friend Derived;

    Derived& derived() {
        return *static_cast<Derived*>(this);
    }

    const Derived& derived() const {
        return *static_cast<const Derived*>(this);
    }

};

class Impl : public Base<Impl> {
public:
    
    Impl(int val)
        : Base(val) {
        
    }
    
    int implementation(int val) {
        return val * val;
    }
};

''')

True

In [7]:
cppyy.gbl.Impl(3).interface()

9

In [7]:
gbl.TemplateOverloadedThrows[int]().throws(1.)

TypeError: none of the 2 overloaded methods succeeded. Full details:
  void TemplateOverloadedThrows<int>::throws(int i) =>
    TypeError: could not convert argument 1 (int/long conversion expects an integer object)
  void TemplateOverloadedThrows<int>::throws(double i) =>
    Exception: std::exception (C++ exception)

In [8]:
gbl.TemplateOverloadedThrows[int]().throws(1)

Exception: none of the 2 overloaded methods succeeded. Full details:
  void TemplateOverloadedThrows<int>::throws(int i) =>
    Exception: std::exception (C++ exception)
  void TemplateOverloadedThrows<int>::throws(double i) =>
    Exception: std::exception (C++ exception)

In [4]:
gbl.TemplateThrows[int]().throws(1)

Exception: void TemplateThrows<int>::throws(int i) =>
    Exception: std::exception (C++ exception)

In [9]:
try:
    gbl.OverloadedThrows().throws(1.)
except Exception as e:
    print(e)
    print(dir(e))
    print(e.args)
    print(type(e))

none of the 2 overloaded methods succeeded. Full details:
  void OverloadedThrows::throws(int i) =>
    TypeError: could not convert argument 1 (int/long conversion expects an integer object)
  void OverloadedThrows::throws(double i) =>
    Exception: std::exception (C++ exception)
['__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', 'args', 'with_traceback']
('none of the 2 overloaded methods succeeded. Full details:\n  void OverloadedThrows::throws(int i) =>\n    TypeError: could not convert argument 1 (int/long conversion expects an integer object)\n  void OverloadedThrows::throws(double i) =>\n    Exception: std::exception (C++ exception)',

In [6]:
gbl.RegularThrows().throws()

Exception: void RegularThrows::throws() =>
    Exception: std::exception (C++ exception)

In [10]:
s = 'AAAGTGTAGAAAAAAAAAAAAAAAAAAAAAGGGTTGGGGGGGGGTTT'

In [11]:
G = std.make_shared[libboink.dBG[libboink.storage.SparseppSetStorage, libboink.hashing.RollingHashShifter]](31)

In [12]:
instrumentation = Instrumentation('', expose=False)

In [13]:
cptor = libboink.cdbg.StreamingCompactor[type(G)].Compactor.build(G, instrumentation.Registry)

In [14]:
G.insert_sequence(s)

17

In [15]:
asm = Assembler(G)

In [18]:
seed = std.string(s[:30])

In [19]:
asm.traverser.set_cursor(seed)

TypeError: cannot resolve method template call for 'set_cursor'

In [9]:
seq,  (r_state, r_end_hash) = asm.assemble_right(s[:31])

In [10]:
seq

'AAAAAAAAAAAAAAAGAAAGTGTAGAAAAAAGGTTGGGGGGGGGTTT'

In [13]:
state == libboink.TraversalState.STOP_FWD

True

In [24]:
act[100:200]

'TGTCGGGCAAGAGAAGTGCTCGCCATAAACGAGGGTGGACCTTTTCCTCCATGTTGGGGACAACTGCGCCGTCTGGGACCGGTAGAGTAAGGTCCCGCTC'

In [25]:
exp[100:200]

'TGTCGGGCAAGAGAAGTGCTCGCCATAAACGAGGGTGGACCTTTTCCTCCGCCGATGTTGGGGACAACTGCTCTGGGACCGGTAGAGTAAGGTCCCGCTC'

In [13]:
act = 'CCTCGGGATTGCATAGGTAAAATAGCATTAGCCAGCAGTTCGGCCTCGGAGCACCCTGTCACCCGCAAGGGCCATCGGTACGACAGGCAGCCGTAGTTATTGTCGGGCAAGAGAAGTGCTCGCCATAAACGAGGGTGGACCTTTTCCTCCATGTTGGGGACAACTGCGCCGTCTGGGACCGGTAGAGTAAGGTCCCGCTCCGGATATCGACGTCTGGGAAACGAGCCGCGTTACGGGTCTCCAATTTCGAACATCACATCACGAGTCGCTACCCTCATGAGCGCAACGGAGATGGTCGGATAGGAACCACAACTCTGCAGAAGTGAATTTATCAGTCGCCTAGACATATGTATTGCAGACTAAATCTGCTAGGTAAACTACCATGTTGCTCATGCACACGAATCATCGAAAAAATGGCGGAACTGTAAACAAGGTCCGCTTGTTTGCAAAATGTACCGGTCTAGCCGGCCCGAAGTCTCCCGATCGAAGTAATTAGAGCT'

exp = 'CCTCGGGATTGCATAGGTAAAATAGCATTAGCCAGCAGTTCGGCCTCGGAGCACCCTGTCACCCGCAAGGGCCATCGGTACGACAGGCAGCCGTAGTTATTGTCGGGCAAGAGAAGTGCTCGCCATAAACGAGGGTGGACCTTTTCCTCCGCCGATGTTGGGGACAACTGCTCTGGGACCGGTAGAGTAAGGTCCCGCTCCGGATATCGACGTCTGGGAAACGAGCCGCGTTACGGGTCTCCAATTTCGAACATCACATCACGAGTCGCTACCCTCATGAGCGCAACGGAGATGGTCGGATAGGAACCACAACTCTGCAGAAGTGAATTTATCAGTCGCCTAGACATATGTATTGCAGACTAAATCTGCTAGGTAAACTACCATGTTGCTCATGCACACGAATCATCGAAAAAATGGCGGAACTGTAAACAAGGTCCGCTTGTTTGCAAAATGTACCGGTCTAGCCGGCCCGAAGTCTCCCGATCGAAGTAATTAGAGCT'

In [26]:
print('\n'.join(difflib.ndiff(act[100:200], exp[100:200])))

  T
  G
  T
  C
  G
  G
  G
  C
  A
  A
  G
  A
  G
  A
  A
  G
  T
  G
  C
  T
  C
  G
  C
  C
  A
  T
  A
  A
  A
  C
  G
  A
  G
  G
  G
  T
  G
  G
  A
  C
  C
  T
  T
  T
  T
  C
  C
  T
  C
  C
+ G
+ C
+ C
+ G
  A
  T
  G
  T
  T
  G
  G
  G
  G
  A
  C
  A
  A
  C
  T
  G
  C
- G
- C
- C
- G
  T
  C
  T
  G
  G
  G
  A
  C
  C
  G
  G
  T
  A
  G
  A
  G
  T
  A
  A
  G
  G
  T
  C
  C
  C
  G
  C
  T
  C


In [5]:
libboink.TraversalState.

<namespace boink.boink.TraversalState at 0x556fd5040268>

In [10]:
state == libboink.Traverse[type(G)].State.STOP_FWD

AttributeError: type object 'int' has no attribute 'STOP_FWD'

In [21]:
libboink.Traverse[type(G)].State.STOP_FWD

AttributeError: type object 'State' has no attribute 'STOP_FWD'

In [8]:
G.right_degree(s)

1

In [8]:
G.clone()

<boink.boink.dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter> object at 0x55ae38767560 held by shared_ptr<boink::dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter> > at 0x55ae34f29c00>

In [2]:
storage_t = libboink.storage.BitStorage
signature_t = libboink.signatures.UnikmerSignature[storage_t]

In [3]:
ukhs = load_ukhs(31, 8)

In [4]:
sig = signature_t.Signature.build(31, 8, ukhs.__smartptr__(), 10000, 4)

In [5]:
proc = std.make_shared[signature_t.Processor](sig.__smartptr__(), 10000, 100000, 100000)

In [6]:
proc

<boink.boink.signatures.UnikmerSignature<boink::storage::BitStorage>.UnikmerSignature<boink::storage::BitStorage>::Processor object at 0x55c72107e5e0 held by shared_ptr<boink::signatures::UnikmerSignature<boink::storage::BitStorage>::Processor> at 0x55c71d57fd70>

In [16]:
type(G)

<class boink.boink.dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter> at 0x55ed277bb6c8>

In [12]:
std.pair[std.vector[libboink.hashing.kmer_t], std.vector[libboink.hashing.kmer_t]]

<class cppyy.gbl.std.pair<vector<boink::hashing::kmer_t>,vector<boink::hashing::kmer_t> > at 0x55ad9f2d2968>

In [6]:
parser = cppyy.gbl.ParserFactory[boink.parsing.FastxReader].build('/work/draff/results/simulated/ecoli.0._100_180_1.fq')

In [56]:
path = boink.Path()

In [44]:
shifter = boink.hashing.RollingHashShifter(7)

In [58]:
for i in range(5):
    path.clear()
    shifter.get_cursor(path)
    print(asm.to_string(path))

GGATTCG
GGATTCG
GGATTCG
GGATTCG
GGATTCG


In [45]:
for i in range(len(s) - 9 + 1):
    kmer = s[i:i+9]
    shifter.set_cursor(kmer) 
    print(shifter.get(), shifter.hash(kmer))
    #print(kmer, int(asm.degree_left()), int(asm.degree_right()))

16928099823363697171 16928099823363697171
17774349600126154739 17774349600126154739
5432234656955854598 5432234656955854598
15864785342952959926 15864785342952959926
807854437867792248 807854437867792248
503930369671869761 503930369671869761
14846359574385085156 14846359574385085156
6910144570189067695 6910144570189067695
8953154543898151736 8953154543898151736
15784469927269168548 15784469927269168548
12000143714024755448 12000143714024755448
10045950146635431439 10045950146635431439
3219486363132302806 3219486363132302806
11146315393402538954 11146315393402538954
17774349600126154739 17774349600126154739
5432234656955854598 5432234656955854598
15864785342952959926 15864785342952959926
807854437867792248 807854437867792248
503930369671869761 503930369671869761
14846359574385085156 14846359574385085156


In [49]:
shifter = boink.hashing.RollingHashShifter('AAAAAAA', 7)

In [26]:
shifter.update_right('C')

17896415738016705498

In [27]:
shifter.get()

17896415738016705498

In [16]:
storages = ['BitStorage', 'ByteStorage', 'NibbleStorage', 'QFStorage', 'SparseppSetStorage']

def storage_decls(template):

    for storage in storages:
        print(template.format(storage))
    print()
    template = 'extern\n' + template
    for storage in storages:
        print(template.format(storage))

In [10]:
storage_decls('template class boink::dBG<boink::storage::{},\n'
              '                          boink::hashing::RollingHashShifter>;')

template class boink::dBG<boink::storage::BitStorage,
                          boink::hashing::RollingHashShifter>;
template class boink::dBG<boink::storage::ByteStorage,
                          boink::hashing::RollingHashShifter>;
template class boink::dBG<boink::storage::NibbleStorage,
                          boink::hashing::RollingHashShifter>;
template class boink::dBG<boink::storage::QFStorage,
                          boink::hashing::RollingHashShifter>;
template class boink::dBG<boink::storage::SparseppSetStorage,
                          boink::hashing::RollingHashShifter>;

extern
template class boink::dBG<boink::storage::BitStorage,
                          boink::hashing::RollingHashShifter>;
extern
template class boink::dBG<boink::storage::ByteStorage,
                          boink::hashing::RollingHashShifter>;
extern
template class boink::dBG<boink::storage::NibbleStorage,
                          boink::hashing::RollingHashShifter>;
extern
template class boink

In [11]:
storage_decls('template class boink::cdbg::cDBG<boink::dBG<boink::storage::{},\n'
              '                                            boink::hashing::RollingHashShifter>>;')

template class boink::cdbg::cDBG<boink::dBG<boink::storage::BitStorage,
                                            boink::hashing::RollingHashShifter>>;
template class boink::cdbg::cDBG<boink::dBG<boink::storage::ByteStorage,
                                            boink::hashing::RollingHashShifter>>;
template class boink::cdbg::cDBG<boink::dBG<boink::storage::NibbleStorage,
                                            boink::hashing::RollingHashShifter>>;
template class boink::cdbg::cDBG<boink::dBG<boink::storage::QFStorage,
                                            boink::hashing::RollingHashShifter>>;
template class boink::cdbg::cDBG<boink::dBG<boink::storage::SparseppSetStorage,
                                            boink::hashing::RollingHashShifter>>;

extern
template class boink::cdbg::cDBG<boink::dBG<boink::storage::BitStorage,
                                            boink::hashing::RollingHashShifter>>;
extern
template class boink::cdbg::cDBG<boink::dBG<boink:

In [15]:
storage_decls('template class boink::PdBG<boink::storage::{}>;')

template class boink::PdBG<boink::storage::BitStorage>;
template class boink::PdBG<boink::storage::ByteStorage>;
template class boink::PdBG<boink::storage::NibbleStorage>;
template class boink::PdBG<boink::storage::QFStorage>;
template class boink::PdBG<boink::storage::SparseppSetStorage>;

extern
template class boink::PdBG<boink::storage::BitStorage>;
extern
template class boink::PdBG<boink::storage::ByteStorage>;
extern
template class boink::PdBG<boink::storage::NibbleStorage>;
extern
template class boink::PdBG<boink::storage::QFStorage>;
extern
template class boink::PdBG<boink::storage::SparseppSetStorage>;


In [17]:
storage_decls('template struct boink::Traverse<boink::dBG<boink::storage::{},\n'
          '                                               boink::hashing::RollingHashShifter>>;')

template struct boink::Traverse<boink::dBG<boink::storage::BitStorage,
                                               boink::hashing::RollingHashShifter>>;
template struct boink::Traverse<boink::dBG<boink::storage::ByteStorage,
                                               boink::hashing::RollingHashShifter>>;
template struct boink::Traverse<boink::dBG<boink::storage::NibbleStorage,
                                               boink::hashing::RollingHashShifter>>;
template struct boink::Traverse<boink::dBG<boink::storage::QFStorage,
                                               boink::hashing::RollingHashShifter>>;
template struct boink::Traverse<boink::dBG<boink::storage::SparseppSetStorage,
                                               boink::hashing::RollingHashShifter>>;

extern
template struct boink::Traverse<boink::dBG<boink::storage::BitStorage,
                                               boink::hashing::RollingHashShifter>>;
extern
template struct boink::Traverse<boink:

In [23]:
storage_decls('template class boink::FileConsumer<boink::dBG<boink::storage::{},\n'
              '                                              boink::hashing::RollingHashShifter>>;')

template class boink::FileConsumer<boink::dBG<boink::storage::BitStorage,
                                              boink::hashing::RollingHashShifter>>;
template class boink::FileConsumer<boink::dBG<boink::storage::ByteStorage,
                                              boink::hashing::RollingHashShifter>>;
template class boink::FileConsumer<boink::dBG<boink::storage::NibbleStorage,
                                              boink::hashing::RollingHashShifter>>;
template class boink::FileConsumer<boink::dBG<boink::storage::QFStorage,
                                              boink::hashing::RollingHashShifter>>;
template class boink::FileConsumer<boink::dBG<boink::storage::SparseppSetStorage,
                                              boink::hashing::RollingHashShifter>>;

extern
template class boink::FileConsumer<boink::dBG<boink::storage::BitStorage,
                                              boink::hashing::RollingHashShifter>>;
extern
template class boink::File

In [24]:
storage_decls('template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::{},\n'
              '                                                       boink::hashing::RollingHashShifter>>;')

template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::BitStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::ByteStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::NibbleStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::QFStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::SparseppSetStorage,
                                                       boink::hashing::RollingHashShifter>>;

extern
template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::BitStorage,
                  

In [26]:
storage_decls('template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::{},\n'
              '                                               boink::hashing::RollingHashShifter>>;')

template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::BitStorage,
                                               boink::hashing::RollingHashShifter>>;
template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::ByteStorage,
                                               boink::hashing::RollingHashShifter>>;
template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::NibbleStorage,
                                               boink::hashing::RollingHashShifter>>;
template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::QFStorage,
                                               boink::hashing::RollingHashShifter>>;
template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::SparseppSetStorage,
                                               boink::hashing::RollingHashShifter>>;

extern
template class boink::cdbg::StreamingCompactor<boink::dBG<boink::storage::BitStorage,
                                        

In [27]:
storage_decls('template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::{},\n'
              '                                                             boink::hashing::RollingHashShifter>>;')

template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::BitStorage,
                                                             boink::hashing::RollingHashShifter>>;
template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::ByteStorage,
                                                             boink::hashing::RollingHashShifter>>;
template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::NibbleStorage,
                                                             boink::hashing::RollingHashShifter>>;
template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::QFStorage,
                                                             boink::hashing::RollingHashShifter>>;
template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::SparseppSetStorage,
                                                             boink::hashing::RollingHashShifter>>;

extern
template class boink::StreamingCompactorP

In [29]:
storage_decls('template class boink::signatures::UnikmerSignature<boink::storage::{}>;')

template class boink::signatures::UnikmerSignature<boink::storage::BitStorage>;
template class boink::signatures::UnikmerSignature<boink::storage::ByteStorage>;
template class boink::signatures::UnikmerSignature<boink::storage::NibbleStorage>;
template class boink::signatures::UnikmerSignature<boink::storage::QFStorage>;
template class boink::signatures::UnikmerSignature<boink::storage::SparseppSetStorage>;

extern
template class boink::signatures::UnikmerSignature<boink::storage::BitStorage>;
extern
template class boink::signatures::UnikmerSignature<boink::storage::ByteStorage>;
extern
template class boink::signatures::UnikmerSignature<boink::storage::NibbleStorage>;
extern
template class boink::signatures::UnikmerSignature<boink::storage::QFStorage>;
extern
template class boink::signatures::UnikmerSignature<boink::storage::SparseppSetStorage>;


In [16]:
storage_decls('template class std::enable_shared_from_this<boink::dBG<boink::storage::{},\n'
              '                                                       boink::hashing::RollingHashShifter>>;')

template class std::enable_shared_from_this<boink::dBG<boink::storage::BitStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class std::enable_shared_from_this<boink::dBG<boink::storage::ByteStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class std::enable_shared_from_this<boink::dBG<boink::storage::NibbleStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class std::enable_shared_from_this<boink::dBG<boink::storage::QFStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class std::enable_shared_from_this<boink::dBG<boink::storage::SparseppSetStorage,
                                                       boink::hashing::RollingHashShifter>>;

extern
template class std::enable_shared_from_this<boink::dBG<boink::storage::BitStorage,
                  

In [15]:
storage_decls('template class std::enable_shared_from_this<boink::PdBG<boink::storage::{}>>;')


template class std::enable_shared_from_this<boink::PdBG<boink::storage::BitStorage>>;
template class std::enable_shared_from_this<boink::PdBG<boink::storage::ByteStorage>>;
template class std::enable_shared_from_this<boink::PdBG<boink::storage::NibbleStorage>>;
template class std::enable_shared_from_this<boink::PdBG<boink::storage::QFStorage>>;
template class std::enable_shared_from_this<boink::PdBG<boink::storage::SparseppSetStorage>>;

extern
template class std::enable_shared_from_this<boink::PdBG<boink::storage::BitStorage>>;
extern
template class std::enable_shared_from_this<boink::PdBG<boink::storage::ByteStorage>>;
extern
template class std::enable_shared_from_this<boink::PdBG<boink::storage::NibbleStorage>>;
extern
template class std::enable_shared_from_this<boink::PdBG<boink::storage::QFStorage>>;
extern
template class std::enable_shared_from_this<boink::PdBG<boink::storage::SparseppSetStorage>>;
